# Árboles de decisión y el algoritmo CART

## 1. Introducción

Los **árboles de decisión** constituyen uno de los métodos más interpretables y ampliamente utilizados en el aprendizaje automático supervisado.  
Su estructura jerárquica permite dividir progresivamente el espacio de atributos en regiones homogéneas respecto a la variable de salida, generando reglas de decisión de fácil comprensión.  

Cada división (o *split*) en el árbol se selecciona con el objetivo de reducir al máximo la heterogeneidad de las clases en los nodos hijos.  
Para lograrlo, se emplean métricas de **impureza**, como la **entropía** o el **índice de Gini**.  



## 2. Medidas de impureza: Entropía vs. Gini

- **Entropía** (usada en ID3/C4.5):
  $$
  H(t) = -\sum_{j=1}^K p(j\mid t)\,\log_2 p(j\mid t)
  $$

  mide el desorden en el nodo. Valores altos indican una distribución equilibrada de clases.

- **Índice de Gini** (usado en CART):  
  $$
  H(t) = -\sum_{j=1}^K p(j\mid t)\,\log_2 p(j\mid t)
  $$

  mide la probabilidad de clasificación errónea al asignar etiquetas al azar según la distribución de clases.  

Ambas métricas son coherentes y producen resultados similares en la práctica.  
Sin embargo, el índice de Gini es **computacionalmente más eficiente**, al no requerir el cálculo de logaritmos, y tiende a generar divisiones algo más equilibradas en cuanto al tamaño de los nodos.  
Por estas razones, **CART adopta Gini como medida por defecto**.


## 3. De binario a multiclase

En sus orígenes, los árboles de decisión se aplicaron principalmente a problemas de **clasificación binaria**.  
No obstante, la definición del índice de Gini es lo suficientemente general para extenderse de manera inmediata al caso de **múltiples clases**:

$$
\text{Gini}(t) = 1 - \sum_{j=1}^K p(j \mid t)^2, \quad K \geq 2
$$


De esta forma, la estructura binaria del árbol (dos nodos hijos por división) se mantiene, pero los nodos pueden contener observaciones de varias clases simultáneamente.  
El criterio de división sigue siendo válido en cualquier escenario: minimizar la impureza ponderada de los nodos hijos.



## 4. El algoritmo CART

El método **CART (Classification and Regression Trees)** formaliza este proceso en cuatro pasos principales:


### 1. **Selección de variable y umbral**

Para cada atributo, se ordenan sus valores y se prueban como candidatos de corte los puntos medios entre observaciones consecutivas distintas:

$$
c_i = \frac{x_{(i)} + x_{(i+1)}}{2}, \quad i = 1, \dots, n-1
$$


### 2. **Evaluación del split**

Cada corte induce una partición \((t_L, t_R)\).  
La calidad del split se evalúa mediante la impureza ponderada:

$$
I(s,t) = \frac{n_L}{n}\,\text{Gini}(t_L) + \frac{n_R}{n}\,\text{Gini}(t_R)
$$


### 3. **Elección del mejor split**

El par (atributo, umbral) que minimiza la impureza se selecciona como división óptima:

$$
s^* = \arg\min_{s}\; I(s,t)
$$


### 4. **Recursividad y detención**

El proceso se repite en los nodos hijos hasta cumplir un criterio de parada: nodo puro, tamaño mínimo de muestras o profundidad máxima.


## Búsqueda de umbral en variables continuas

Para variables numéricas, CART **ordena los valores de la característica** y evalúa cortes posibles entre observaciones consecutivas distintas.  
Cada corte se coloca en el **punto medio** entre dos valores distintos.

$$
c = \frac{x_{(i)} + x_{(i+1)}}{2}, 
\quad \text{con } n-1 \text{ posibles cortes entre valores consecutivos}
$$

- Si los valores son iguales, no se prueba un corte (no aporta nada).  
- Si los valores difieren, el corte se evalúa justo en el medio.  



### Clase `Nodo`

Representa un nodo del árbol, ya sea interno o una hoja.  
Contiene la información local y los enlaces a subárboles.

- **Atributos principales:**
  - `X`, `Y`: subconjunto de datos y etiquetas en el nodo.
  - `feature_index`, `threshold`: variable y umbral elegidos para dividir.
  - `left`, `right`: nodos hijos izquierdo y derecho.
  - `label`: clase mayoritaria cuando el nodo es hoja.

- **Funciones clave:**
  - `is_terminal()`: determina si el nodo debe ser una hoja  
    (cuando no hay datos o todas las observaciones son de la misma clase).
  - `gini_counts(counts, n)`: calcula la impureza de Gini de un nodo a partir de conteos de clases:  
    $$
    \text{Gini}(t) = 1 - \sum_{j=1}^K p(j \mid t)^2
    $$

  - `best_split(min_samples_leaf)`: busca la mejor división en el nodo actual.  
    - Ordena los valores de cada característica.  
    - Evalúa cortes en los puntos medios entre valores consecutivos distintos.  
    - Calcula la **impureza ponderada**:  
      $$
      I(s,t) = \frac{n_L}{n}\,\text{Gini}(t_L) + \frac{n_R}{n}\,\text{Gini}(t_R)
      $$
    - Devuelve la mejor variable y umbral encontrados.


In [ ]:
class Nodo:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.left = None
        self.right = None
        self.feature_index = None
        self.threshold = None
        self.label = None

    def is_terminal(self):
        return len(self.Y) == 0 or len(set(self.Y)) == 1

    def gini_counts(self, counts, n):
        # counts: vector con conteos por clase (long K)
        if n == 0:
            return 0.0
        p = counts / n
        return 1.0 - np.sum(p * p)

    def best_split(self, min_samples_leaf=1):
        n, d = self.X.shape
        if n <= 1:
            return None

        # Mapear etiquetas a 0..K-1 una sola vez por nodo
        classes, y_idx = np.unique(self.Y, return_inverse=True)
        K = len(classes)

        best_gini = np.inf
        best = None

        # Conteos totales a la derecha (antes de partir)
        total_right = np.bincount(y_idx, minlength=K).astype(np.int64)

        for j in range(d):
            # Ordenar por la columna j
            order = np.argsort(self.X[:, j], kind="mergesort")
            xj = self.X[order, j]
            yj = y_idx[order]

            left_counts = np.zeros(K, dtype=np.int64)
            right_counts = total_right.copy()

            # Recorremos posibles cortes entre i e i+1
            for i in range(n - 1):
                c = yj[i]
                left_counts[c]  += 1
                right_counts[c] -= 1

                # Saltar si los valores son iguales 
                if xj[i] == xj[i + 1]:
                    continue

                nL = i + 1
                nR = n - nL
                # hojas mínimas
                if nL < min_samples_leaf or nR < min_samples_leaf:
                    continue

                gL = self.gini_counts(left_counts, nL)
                gR = self.gini_counts(right_counts, nR)
                g  = (nL * gL + nR * gR) / n  # PONDERADO

                if g < best_gini:
                    thr = (xj[i] + xj[i + 1]) / 2.0  # punto medio
                    best_gini = g
                    best = (j, thr)

        return best
        


### Clase `DT` (Decision Tree)

Gestiona el árbol completo y su construcción.

- **Parámetros de control:**
  - `max_depth`: profundidad máxima permitida.
  - `min_samples_split`: mínimo de observaciones para intentar un split.
  - `min_samples_leaf`: mínimo de observaciones en una hoja.

- **Funciones clave:**
  - `fit(X, Y)`: inicia el entrenamiento construyendo la raíz con todos los datos.
  - `_grow(X, Y, depth)`: construcción recursiva del árbol.  
    - Evalúa criterios de parada (nodo terminal, profundidad máxima, tamaño mínimo).  
    - Si no se detiene, aplica `best_split()` del nodo actual.  
    - Divide el dataset en izquierdo y derecho, y llama recursivamente.  
  - `_predict_one(node, x)`: predice la clase de una observación recorriendo el árbol desde la raíz hasta una hoja.
  - `predict(X)`: aplica `_predict_one` a todas las observaciones.


In [5]:
class DT:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf

    def fit(self, X, Y):
        self.root = self._grow(X, Y, depth=0)

    def _grow(self, X, Y, depth):
        node = Nodo(X, Y)

        # parada
        if (node.is_terminal() or
            (self.max_depth is not None and depth >= self.max_depth) or
            len(Y) < self.min_samples_split):
            node.label = Counter(Y).most_common(1)[0][0] if len(Y) else None
            return node

        split = node.best_split(min_samples_leaf=self.min_samples_leaf)
        if split is None:
            node.label = Counter(Y).most_common(1)[0][0]
            return node

        j, t = split
        node.feature_index, node.threshold = j, t

        left_mask  = X[:, j] <= t
        right_mask = ~left_mask

        node.left  = self._grow(X[left_mask],  Y[left_mask],  depth + 1)
        node.right = self._grow(X[right_mask], Y[right_mask], depth + 1)
        return node

    def _predict_one(self, node, x):
        if node.label is not None or node.is_terminal():
            return node.label
        if x[node.feature_index] <= node.threshold:
            return self._predict_one(node.left, x)
        else:
            return self._predict_one(node.right, x)

    def predict(self, X):
        return np.array([self._predict_one(self.root, x) for x in X])



### Conexión con la teoría de CART

1. **Medida de impureza:**  
   Se implementa con `gini_counts`, usando la fórmula clásica de Gini.

2. **Selección de split:**  
   `best_split` examina todas las variables y candidatos de corte.  
   Elige la división que **minimiza la impureza ponderada**.

3. **Construcción recursiva:**  
   `_grow` refleja la naturaleza recursiva del algoritmo: cada nodo se expande hasta cumplir un criterio de detención.

4. **Predicción:**  
   `_predict_one` y `predict` reproducen el proceso de clasificación: un ejemplo desciende por el árbol comparando sus atributos con los umbrales hasta llegar a una hoja.


### Resumen

- **`Nodo`** encapsula la lógica local (datos, pureza, mejor división).  
- **`DT`** organiza el crecimiento global del árbol y el proceso de predicción.  
- El flujo completo replica el **algoritmo CART con índice de Gini**, extendido naturalmente al caso multiclase, pues la suma en el cálculo de Gini incluye todas las clases posibles.  
